<a href="https://colab.research.google.com/github/Nisha40108/ParkNaviGo/blob/main/Annotation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
import json
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [69]:
# Load JSON file with annotations
json_file_path = '/content/drive/MyDrive/nisha/train/_groundtruth.json'
with open(json_file_path, 'r') as file:
    data = json.load(file)

    print(data)

[{'filename': 'Screen-Shot-2023-11-29-at-3-40-28-PM_png.rf.03fe551fa68cdcd812156d0bbb9ca25a.jpg', 'label': 'non-empty', 'left': 230, 'top': 196, 'bottom': 265, 'right': 279}, {'filename': 'Screen-Shot-2023-11-29-at-3-40-28-PM_png.rf.03fe551fa68cdcd812156d0bbb9ca25a.jpg', 'label': 'non-empty', 'left': 343, 'top': 249, 'bottom': 331, 'right': 383}, {'filename': 'Screen-Shot-2023-11-29-at-3-40-28-PM_png.rf.03fe551fa68cdcd812156d0bbb9ca25a.jpg', 'label': 'non-empty', 'left': 171, 'top': 183, 'bottom': 243, 'right': 223}, {'filename': 'Screen-Shot-2023-11-29-at-3-40-28-PM_png.rf.03fe551fa68cdcd812156d0bbb9ca25a.jpg', 'label': 'non-empty', 'left': 157, 'top': 180, 'bottom': 234, 'right': 197}, {'filename': 'Screen-Shot-2023-11-29-at-3-40-28-PM_png.rf.03fe551fa68cdcd812156d0bbb9ca25a.jpg', 'label': 'non-empty', 'left': 137, 'top': 184, 'bottom': 230, 'right': 174}, {'filename': 'Screen-Shot-2023-11-29-at-3-40-28-PM_png.rf.03fe551fa68cdcd812156d0bbb9ca25a.jpg', 'label': 'non-empty', 'left': 12

In [66]:
# Convert the data to a pandas DataFrame
annotations_df = pd.DataFrame(data)

In [70]:
# Encode the class labels
label_encoder = LabelEncoder()
annotations_df['class_encoded'] = label_encoder.fit_transform(annotations_df['label'])

In [71]:
# Split data into training and testing sets
train_df, test_df = train_test_split(annotations_df, test_size=0.2, random_state=42)

In [83]:
# Define a function to load images and annotations
def load_images_and_annotations(df):
    data = []
    for _, row in df.iterrows():
        image_path = '/content/drive/MyDrive/nisha/train/Picture1.jpg' + row['filename']  # Adjust the path accordingly
        image = cv2.imread(image_path)
        if image is not None:
          image = image / 255.0

        annotation = {
            'image': image,
            'class_encoded': row['class_encoded'],
            'left': row['left'],
            'top': row['top'],
            'bottom': row['bottom'],
            'right': row['right']
        }
        data.append(annotation)
    return data

In [84]:
# Load training and testing images and annotations
train_data = load_images_and_annotations(train_df)
test_data = load_images_and_annotations(test_df)

In [85]:
# Extract image data and annotations from the loaded data
X_train = np.array([item['image'] for item in train_data])
y_train = np.array([item['class_encoded'] for item in train_data])

X_test = np.array([item['image'] for item in test_data])
y_test = np.array([item['class_encoded'] for item in test_data])


In [86]:
img_width, img_height = 640, 640
input_shape = (img_height, img_width, 3)  # Assuming RGB images



In [87]:
# Define the CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))  # Output layer for binary classification


In [88]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [89]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)


In [90]:
# Assuming your 'filename' column contains the image filenames
# Make sure the 'class_encoded' column contains the class labels

# Define the data generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename',
    y_col='class_encoded',
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='raw'  # Use 'raw' when the target variable is numeric
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filename',
    y_col='class_encoded',
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='raw'  # Use 'raw' when the target variable is numeric
)


Found 0 validated image filenames.
Found 0 validated image filenames.


/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 10 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 3 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(
